<a href="https://colab.research.google.com/github/xsoulsky/mapperatorinator/blob/main/colab/mapperatorinator_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/OliBomby/Mapperatorinator/blob/main/colab/mapperatorinator_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Beatmap Generation with Mapperatorinator

This notebook is an interactive demo of an osu! beatmap generation model created by OliBomby. This model is capable of generating hit objects, hitsounds, timing, kiai times, and SVs for all 4 gamemodes. You can upload a beatmap to give to the model as additional context or remap parts of the beatmap.

### Instructions for running:

* Read and accept the rules regarding using this tool
* Make sure to use a GPU runtime, click:  __Runtime >> Change Runtime Type >> GPU__
* Press ▶️ on the left of each cell to execute the cell
* __Upload Audio__, choose a .mp3 or .ogg file from your computer
* __Upload Beatmap__, optionally choose a .osu file from your computer
* __Configure__, choose your generation parameters to control the style of the generated beatmap
* Generate the beatmap using the __Generate Beatmap__ cell (it may take a few minutes depending on the length of the song)


In [2]:
#@title Setup Environment { display-mode: "form" }
#@markdown ### Use this tool responsibly. Always disclose the use of AI in your beatmaps. Do not upload the generated beatmaps.
i_accept_the_rules = True # @param {type:"boolean"}

assert i_accept_the_rules, "Read and accept the rules first!"

!git clone https://github.com/OliBomby/Mapperatorinator.git
%cd Mapperatorinator

!pip install -r requirements.txt

from google.colab import files

import os
import glob
from hydra import compose, initialize
from osuT5.osuT5.event import ContextType
from inference import main

output_path = "output"
input_audio = ""
input_beatmap = ""

fatal: destination path 'Mapperatorinator' already exists and is not an empty directory.
/content/Mapperatorinator
  Cloning https://github.com/OliBomby/slider.git (to revision gedagedigedagedaoh) to /tmp/pip-install-eljqouhb/slider_575311ddc0184e9eadaa1034fa9ae7f3
  Running command git clone --filter=blob:none --quiet https://github.com/OliBomby/slider.git /tmp/pip-install-eljqouhb/slider_575311ddc0184e9eadaa1034fa9ae7f3
  Running command git checkout -b gedagedigedagedaoh --track origin/gedagedigedagedaoh
  Switched to a new branch 'gedagedigedagedaoh'
  Branch 'gedagedigedagedaoh' set up to track remote branch 'gedagedigedagedaoh' from 'origin'.
  Resolved https://github.com/OliBomby/slider.git to commit a605ceb9e4b7b640904029c930563e8f2e220023
  Preparing metadata (setup.py) ... done


In [3]:
#@title Upload Audio { display-mode: "form" }
#@markdown This is the song to generate a beatmap for. Please upload a .mp3 or .ogg file.

def upload_audio():
    data = list(files.upload().keys())
    if len(data) > 1:
        print('Multiple files uploaded; using only one.')
    file = data[0]
    if not file.endswith('.mp3') and not file.endswith('.ogg'):
        print('Invalid file format. Please upload a .mp3 or .ogg file.')
        return ""
    return data[0]

input_audio = upload_audio()

Saving audio.mp3 to audio.mp3


In [4]:
#@title (Optional) Upload Beatmap { display-mode: "form" }
#@markdown This step is required if you want to use `in_context` or `add_to_beatmap` to provide additional info to the model.
#@markdown It will also fill in any missing metadata and unknown values in the configuration using info of the reference beatmap.
#@markdown Please upload a **.osu** file. You can find the .osu file in the song folder in stable or by using File > Edit Externally in lazer.
use_reference_beatmap = True # @param {type:"boolean"}

def upload_beatmap():
    data = list(files.upload().keys())
    if len(data) > 1:
        print('Multiple files uploaded; using only one.')
    file = data[0]
    if not file.endswith('.osu'):
        print('Invalid file format. Please upload a .osu file.\nIn stable you can find the .osu file in the song folder (File > Open Song Folder).\nIn lazer you can find the .osu file by using File > Edit Externally.')
        return ""
    return file

if use_reference_beatmap:
    input_beatmap = upload_beatmap()
else:
    input_beatmap = ""

Saving VXSTXLXRDE - Resolution (xs_) [papich].osu to VXSTXLXRDE - Resolution (xs_) [papich].osu


In [5]:
#@title Configure and Generate Beatmap { display-mode: "form" }

#@markdown #### You can input -1 to leave the value unknown.
#@markdown ---
#@markdown This is the game mode to generate a beatmap for.
gamemode = "standard" # @param ["standard", "taiko", "catch the beat", "mania"]
#@markdown This is the Star Rating you want your beatmap to be. It might deviate from this number depending on the song intensity and other configuration.
difficulty = 7 # @param {type:"number"}
#@markdown This is the user ID of the ranked mapper to imitate for mapping style. You can find this in the URL of the mapper's profile.
mapper_id = -1 # @param {type:"integer"}
#@markdown This is the year you want the beatmap to be from. It should be in the range of 2007 to 2023.
year = 2023 # @param {type:"integer"}
#@markdown This is whether you want the beatmap to be hitsounded. This works only for mania mode.
hitsounded = False # @param {type:"boolean"}
#@markdown This is the global slider velocity multiplier for the beatmap.
slider_multiplier = 1.4 # @param {type:"slider", min:0.4, max:3.6, step:0.1}
#@markdown This is the circle size (CS) of the beatmap.
circle_size = 4 # @param {type:"number"}
#@markdown This is the number of keys for the mania beatmap. This works only for mania mode.
keycount = 4 # @param {type:"slider", min:1, max:18, step:1}
#@markdown This is the ratio of hold notes to circles in the beatmap. It should be in the range [0,1]. This works only for mania mode.
hold_note_ratio = -1 # @param {type:"number"}
#@markdown This is the ratio of scroll speed changes to the number of notes. It should be in the range [0,1]. This works only for mania and taiko modes.
scroll_speed_ratio = -1 # @param {type:"number"}
#@markdown These descriptors of the beatmap. Descriptors are used to describe the style of the beatmap. All available descriptors can be found [here](https://omdb.nyahh.net/descriptors/).
descriptor_1 = 'bursts' # @param ["slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
descriptor_2 = 'jump aim' # @param ["slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
descriptor_3 = 'clean' # @param ["slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
#@markdown These are negative descriptors of the beatmap. Negative descriptors are used to describe what the beatmap should not have. These work only when `cfg_scale` is greater than 1.
negative_descriptor_1 = '' # @param ["slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
negative_descriptor_2 = '' # @param ["slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
negative_descriptor_3 = '' # @param ["slider only", "circle only", "collab", "megacollab", "marathon", "gungathon", "multi-song", "variable timing", "accelerating bpm", "time signatures", "storyboard", "storyboard gimmick", "keysounds", "download unavailable", "custom skin", "featured artist", "custom song", "style", "messy", "geometric", "grid snap", "hexgrid", "freeform", "symmetrical", "old-style revival", "clean", "slidershapes", "distance snapped", "iNiS-style", "avant-garde", "perfect stacks", "ninja spinners", "simple", "chaotic", "repetition", "progression", "high contrast", "improvisation", "playfield usage", "playfield constraint", "video gimmick", "difficulty spike", "low sv", "high sv", "colorhax", "tech", "slider tech", "complex sv", "reading", "visually dense", "overlap reading", "alt", "jump aim", "sharp aim", "wide aim", "linear aim", "aim control", "flow aim", "precision", "finger control", "complex snap divisors", "bursts", "streams", "spaced streams", "cutstreams", "stamina", "mapping contest", "tournament custom", "tag", "port"] {allow-input: true}
#@markdown ---
#@markdown If true, the generated beatmap will be exported as a .osz file. Otherwise, it will be exported as a .osu file.
export_osz = True # @param {type:"boolean"}
#@markdown If true, the generated beatmap will be added to the reference beatmap and the reference beatmap will be modified instead of creating a new beatmap. It will also continue any hit objects before the start time in the reference beatmap.
add_to_beatmap = False # @param {type:"boolean"}
#@markdown This is the start time of the beatmap in milliseconds. Use this to constrain the generation to a specific part of the song.
start_time = -1 # @param {type:"integer"}
#@markdown This is the end time of the beatmap in milliseconds. Use this to constrain the generation to a specific part of the song.
end_time = -1 # @param {type:"integer"}
#@markdown This is which additional information to give to the model:
#@markdown - TIMING: Give timing points to the model. This will skip the timing point generation step.
#@markdown - KIAI: Give kiai times to the model. This will skip the kiai time generation step.
#@markdown - MAP: Give hit objects to the model. This will skip the hit object generation step.
#@markdown - GD: Give hit objects of another difficulty in the same mapset to the model (can be a different game mode). It will improve the rhythm accuracy and consistency of the generated beatmap without copying the reference beatmap.
#@markdown - NO_HS: Give hit objects without hitsounds to the model. This will copy the hit objects of the reference beatmap and only add hitsounds to them.
in_context = "[NONE]" # @param ["[NONE]", "[TIMING]", "[TIMING,KIAI]", "[TIMING,KIAI,MAP]", "[GD,TIMING,KIAI]", "[NO_HS,TIMING,KIAI]"]
#@markdown This is the output type of the beatmap. You can choose to either generate everything or only generate timing points.
output_type = "[TIMING,KIAI,MAP,SV]" # @param ["[TIMING,KIAI,MAP,SV]", "[TIMING]"]
#@markdown This is the scale of the classifier-free guidance. A higher scale will make the model more likely to follow the descriptors and mapper style. A high `cfg_scale` or certain combinations of settings can produce unexpected results, so use it with caution.
cfg_scale = 1 # @param {type:"slider", min:1, max:10, step:0.1}
#@markdown This is the temperature of the sampling. A lower temperature will make the model more conservative and map everything like a hard diff. I only recommend lowering this when using `add_to_beatmap` and generating small sections.
temperature = 1 # @param {type:"slider", min:0, max:1, step:0.01}
#@markdown This is the random seed. Change this to sample a different beatmap with the same settings.
seed = -1 # @param {type:"integer"}
#@markdown ---
#@markdown This is the number of beams for beam search for the timing generator. Higher values will result in slightly more accurate timing at the cost of speed.
timer_num_beams = 2 # @param {type:"slider", min:1, max:16, step:1}
#@markdown This is the number of iterations for the timing generator. Higher values will result in slightly more accurate timing at the cost of speed.
timer_iterations = 20 # @param {type:"slider", min:1, max:100, step:1}
#@markdown This is the certainty threshold requirement for BPM changes in the timing generator. Higher values will result in less BPM changes.
timer_bpm_threshold = 0.1 # @param {type:"slider", min:0, max:1, step:0.1}
#@markdown ---

# Get actual parameters
a_gamemode = ["standard", "taiko", "catch the beat", "mania"].index(gamemode)
a_difficulty = None if difficulty == -1 else difficulty
a_mapper_id = None if mapper_id == -1 else mapper_id
a_year = None if year == -1 else year
a_circle_size = None if circle_size == -1 else circle_size
a_hold_note_ratio = None if hold_note_ratio == -1 else hold_note_ratio
a_scroll_speed_ratio = None if scroll_speed_ratio == -1 else scroll_speed_ratio
descriptors = [d for d in [descriptor_1, descriptor_2, descriptor_3] if d != '']
negative_descriptors = [d for d in [negative_descriptor_1, negative_descriptor_2, negative_descriptor_3] if d != '']

a_start_time = None if start_time == -1 else start_time
a_end_time = None if end_time == -1 else end_time
a_in_context = [ContextType(c.lower()) for c in in_context[1:-1].split(',')]
a_output_type = [ContextType(c.lower()) for c in output_type[1:-1].split(',')]
a_seed = None if seed == -1 else seed

# Validate stuff
if any(c in a_in_context for c in [ContextType.TIMING, ContextType.KIAI, ContextType.MAP, ContextType.SV, ContextType.GD, ContextType.NO_HS]) or add_to_beatmap:
    assert os.path.exists(input_beatmap), "Please upload a reference beatmap."
assert os.path.exists(input_audio), "Please upload an audio file."

# Create config
with initialize(version_base="1.1", config_path="configs"):
    conf = compose(config_name="inference_v29")

# Do inference
conf.audio_path = input_audio
conf.output_path = output_path
conf.beatmap_path = input_beatmap
conf.gamemode = a_gamemode
conf.difficulty = a_difficulty
conf.mapper_id = a_mapper_id
conf.year = a_year
conf.hitsounded = hitsounded
conf.slider_multiplier = slider_multiplier
conf.circle_size = a_circle_size
conf.keycount = keycount
conf.hold_note_ratio = a_hold_note_ratio
conf.scroll_speed_ratio = a_scroll_speed_ratio
conf.descriptors = descriptors
conf.negative_descriptors = negative_descriptors
conf.export_osz = export_osz
conf.add_to_beatmap = add_to_beatmap
conf.start_time = a_start_time
conf.end_time = a_end_time
conf.in_context = a_in_context
conf.output_type = a_output_type
conf.cfg_scale = cfg_scale
conf.temperature = temperature
conf.seed = a_seed
conf.timer_num_beams = timer_num_beams
conf.timer_iterations = timer_iterations
conf.timer_bpm_threshold = timer_bpm_threshold

_, result_path, osz_path = main(conf)

if osz_path is not None:
    result_path = osz_path

if conf.add_to_beatmap:
    files.download(result_path)
else:
    files.download(result_path)


Random seed: 52390


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

...

seq len 949


  0%|          | 0/100 [00:00<?, ?it/s]

Generated beatmap saved to output/beatmap9f6faaf291f9446db9fbfb07afde4316.osu
Generated .osz saved to output/beatmap26b64d784ad64abca099c285130c0654.osz


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>